# Análise de sentimentos 

## Introdução
    (Atenção: este notebook não está em uso pois o modelo Catboost é muito pesado e optamos por utilizar o XGBoost. Será guardado para futuros testes com Google Colab)
    
Este notebook tem como objetivo realizar análise de sentimento em comentários de clientes de lojas virtuais, oferecidos pela B2W Digital (grupo responsável pelas lojas Submarino, Shoptime e Americanas.com), classificando-os como "Positivos", "Neutros" ou "Negativos", para compor um painel online de de informações e insights sobre a satisfação desses clientes por estado/região do Brasil, construindo assim, uma ferramenta de análise poderosa.
Para isso, aqui vamos usar uma biblioteca de machine learning (aprendizado de máquina) chamada CatBoost, e especificamente o modelo CatBoostClassifier.
Neste notebook teremos textos explicativos, códigos do projeto realizado e gráficos para análise de resultados.

## Machine Learning
Como dito anteriormente, para realizar nosso aprendizado de máquina e conseguir fazer a análise de sentimento dos comentários, optamos por utilizar o CatBoost. Aqui estão alguns motivos que guiaram a escolha dessa tecnologia:

1- Manuseio automático de variáveis categóricas: o CatBoost lida automaticamente com variáveis categóricas, eliminando a necessidade de pré-processamento manual, como a codificação one-hot.

2- Prevenção de overfitting: utiliza técnicas avançadas, como permutação de dados, para evitar overfitting e melhorar a generalização do modelo.

3- Treinamento eficiente em grandes conjuntos de dados: essa biblioteca é otimizada para treinar modelos rapidamente, mesmo em grandes conjuntos de dados, tornando-o adequado para projetos de grande escala.

4- Algoritmo de gradient boosting robusto: baseado em gradient boosting, o CatBoost é poderoso e eficaz para uma variedade de problemas de aprendizado de máquina.

5- Facilidade de uso e interpretação: possui uma interface simples e oferece ferramentas para interpretar e visualizar modelos, facilitando a compreensão do processo de aprendizado de máquina.


## Base de dados
Está disponível em [b2w-reviews01](https://github.com/americanas-tech/b2w-reviews01) e contém mais de 130 mil linhas de avaliações de clientes sobre produtos adquiridos no site Americanas.com.
Essa base conta com diversas informações, entre as mais importantes para nós estão a avaliação em comentário, a nota que reflete a satisfacação do cliente com base na compra feita e o estado do cliente que avaliou. Além disso também temos mais informações do produto como categoria, marca etc, mais informações pessoais dos clientes como sexo, ano de nascimento etc, um complemento ao nível de satisfação com informação de possível indicação dos produtos, e claro, data das avaliações realizadas.

## Principais Bibliotecas

### Catboost
O CatBoost é uma biblioteca de código aberto para machine learning que lida bem com dados categóricos, sem precisar de muita preparação dos dados. Ele é eficiente e fácil de usar, sendo uma escolha popular para classificação e regressão com dados complexos.

### Pandas
A biblioteca pandas é essencial para a manipulação e análise de dados. Ela oferece estruturas de dados flexíveis, como DataFrames, que são ideais para carregar, manipular e analisar grandes conjuntos de dados. No nosso projeto, pandas é utilizada para ler arquivos CSV, explorar os dados e realizar operações de limpeza e transformação.

### NLTK
O NLTK (Natural Language Toolkit) é uma biblioteca poderosa para processamento de linguagem natural (PLN). Ela inclui ferramentas para tokenização, stemming, lematização e remoção de stopwords, que são fundamentais para a preparação de dados textuais. No nosso contexto de análise de sentimentos, o nltk ajuda a transformar texto bruto em uma forma que pode ser analisada por algoritmos de machine learning.

### sklearn (scikit-learn)
A biblioteca scikit-learn é uma ferramenta robusta para machine learning em Python. Ela oferece vários módulos para dividir dados, criar modelos e avaliar sua performance. Neste projeto, scikit-learn é utilizada para:
* model_selection: nas ferramentas como train_test_split e StratifiedKFold para dividir os dados em conjuntos de treino e teste, e para realizar validação cruzada.
* feature_extraction.text: o CountVectorizer transforma texto em vetores de contagem de palavras, permitindo que dados textuais sejam usados por modelos de machine learning.
* metrics: nas funções como confusion_matrix, precision_score, recall_score, f1_score, roc_curve e auc para avaliar a performance do modelo.

### Matplotlib e seaborn
As bibliotecas matplotlib e seaborn são usadas para visualização de dados. Elas permitem criar gráficos e visualizações que ajudam a entender a distribuição dos dados e a performance do modelo. Especificamente, usamos:
* matplotlib.pyplot: ferramentas básicas para criação de gráficos 2D.
* seaborn: facilita a criação de gráficos estatísticos mais atraentes e informativos, ajudando na análise exploratória dos dados.

### Joblib
Essa biblioteca é utilizada para serializar e desserializar objetos Python, como modelos de machine learning treinados. Isso permite salvar um modelo treinado em disco e carregá-lo posteriormente para fazer previsões.

In [7]:
# Importação das bibliotecas usadas
import re
import string
import unicodedata
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import dump
from tabulate import tabulate
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold


# Baixa recursos necessários do NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ghans\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ghans\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ghans\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Tratamento da base
Aqui veremos uma série de etapas para limpar e tratar nossa base de dados, para que seja possível realizar um treinamento efetivo do modelo com ela.

### Limpeza inicial

Estamos usando a já mencionada biblioteca Pandas, para fazer a leitura da nossa base, que está em formato CSV. Após isso, removemos as linhas com valores nulos nas colunas de comentário (review_text) e nota da avaliação (overall_rating).

In [8]:
# Carrega o conjunto de dados
dataset = pd.read_csv("C:\\fatec\\B2W-Reviews01\\B2W-Reviews01.csv")

# Mostra o tamanho do dataset original
print("Tamanho do dataset original:", dataset.shape[0], "\n")

# Remove linhas com valores em branco
dataset = dataset.dropna(subset=['review_text', 'overall_rating'])

# Mostra o tamanho do dataset alterado
print("Tamanho do dataset alterado:", dataset.shape[0], "\n")


Tamanho do dataset original: 132373 

Tamanho do dataset alterado: 129098 



C:\Users\ghans\AppData\Local\Temp\ipykernel_24396\3311458984.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("C:\\fatec\\B2W-Reviews01\\B2W-Reviews01.csv")


### Classificação de comentários
Agora que limpamos, precisamos classificar os comentários para ter um rótulo dos nossos dados e assim tornar possível o treinamento. Para isso, criamos uma nova coluna chamada 'feeling' (sentimento), que armazenará uma das três opções: 0, 1 e 2, considerando que 0 representa comentários negativos, 1 representa comentários neutros e 2 representa comentários positivos. 
Essa atribuição foi feita com base na nota de cada comentário da avaliação, os comentários com nota abaixo de 3 consideramos como negativos, os com nota igual a 3 são neutros e com nota maior que 2, são positivos. Lembrando que cada comentário pode ter uma nota de 0 a 5.

In [9]:
# Cria uma nova coluna para classificar entre comentários positivos(2), negativos(0) ou neutros(1) com base na nota:
dataset['feeling'] = np.where(dataset['overall_rating'] < 3, 0, np.where(dataset['overall_rating'] == 3, 1, 2))

# Mostra os primeiros registros do dataset
print("Primeiros registros do dataset:")
print(tabulate(dataset.head(20), headers='keys', tablefmt='pipe'))


Primeiros registros do dataset:
|    | submission_date     | reviewer_id                                                      |   product_id | product_name                                                                                                                   | product_brand   | site_category_lv1       | site_category_lv2             | review_title                     |   overall_rating | recommend_to_a_friend   | review_text                                                                                                                                                                                                                                                         |   reviewer_birth_year | reviewer_gender   | reviewer_state   |   feeling |
|---:|:--------------------|:-----------------------------------------------------------------|-------------:|:----------------------------------------------------------------------------------------------------------------------------

### Tratamento do texto
Aqui vamos passar nossa base de dados por mais diversas etapas de limpeza e pré-processamento de dados, para tratar o texto e assim facilitar a obteção de bons resultados no treinamento. As etapas utilizadas foram:

* Conversão para minúsculas:todos os caracteres são convertidos para minúsculas para garantir consistência nos dados.
* Remoção de Acentos: simplifica o texto, removendo caracteres especiais.
* Remoção de Números: os números são removidos usando expressões regulares, pois geralmente não são relevantes para a análise de texto.
* Remoção de Caracteres Especiais: caracteres especiais, como emojis, são removidos para manter apenas o texto relevante.
* Remoção de Pontuação: ajuda a limpar o texto e simplificar a tokenização.
* Remoção de Espaços Extras: espaços extras são removidos para garantir que o texto esteja bem formatado.
* Tokenização: o texto é dividido em tokens (palavras individuais) para facilitar a lematização e a remoção de stopwords.
* **Lematização** e **Remoção de Stopwords**: cada token é lematizado (reduzido à sua forma base) e as stopwords são removidas para melhorar a precisão da análise.
    * A **lematização** é o processo de reduzir uma palavra à sua forma base, chamada lema. Por exemplo, as palavras "gato", "gata", "gatos" e "gatas" têm o mesmo lema: "gato". Isso é útil na análise de texto porque simplifica as palavras, permitindo que os modelos de linguagem entendam melhor o significado dos textos.

    * A **remoção de stopwords** é uma etapa essencial no pré-processamento de texto, pois elimina palavras comuns que não contribuem para a análise, como "e", "de" e "que". Isso ajuda a reduzir o ruído nos dados e a melhorar a eficácia dos modelos de análise de texto.


In [10]:
# Criação da instância do lematizador e das stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

# Lista para armazenar os textos pré-processados
preprocessed_texts = []

# Itera sobre cada texto no dataset para pré-processamento
for text in dataset['review_text']:
    # Verifica se o texto é uma string
    if isinstance(text, str):
        # Converte para minúsculas
        text = text.lower()
        # Remove acentos
        text = ''.join(char for char in unicodedata.normalize('NFKD', text) if unicodedata.category(char) != 'Mn')
        # Remove números usando expressão regular
        text = re.sub(r'\d+', '', text)
        # Remove caracteres especiais (incluindo emojis)
        text = re.sub(r'[^\w\s]', '', text)
        # Remove pontuação
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remove espaços extras
        text = re.sub(r'\s+', ' ', text).strip()
        # Tokenização
        tokens = word_tokenize(text)
        # Lematização e remoção de stopwords
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word.lower() not in stop_words]
        # Junta os tokens em texto novamente
        preprocessed_text = ' '.join(tokens)
        # Adiciona o texto pré-processado à lista
        preprocessed_texts.append(preprocessed_text)
    else:
        preprocessed_texts.append("")

# Substitui os textos originais pelos textos já preparados para análise
dataset['review_text'] = preprocessed_texts

# Mostra a base de dados após o pré-processamento
print("\nDataset após pré-processamento:")
print(tabulate(dataset.head(10), headers='keys', tablefmt='pipe'))


Dataset após pré-processamento:
|    | submission_date     | reviewer_id                                                      |   product_id | product_name                                                                                                                   | product_brand   | site_category_lv1       | site_category_lv2             | review_title                     |   overall_rating | recommend_to_a_friend   | review_text                                                                                                                                                                                        |   reviewer_birth_year | reviewer_gender   | reviewer_state   |   feeling |
|---:|:--------------------|:-----------------------------------------------------------------|-------------:|:-------------------------------------------------------------------------------------------------------------------------------|:----------------|:------------------------|:---------------

In [11]:
# Separar os dados em features (X) e target (Y)
X = dataset['review_text'].values
Y = dataset['feeling'].values

In [12]:
# Criar um objeto CountVectorizer com N-grams
ngram_vectorizer = CountVectorizer(ngram_range=(1, 3))  # Considera unigramas, bigramas e trigramas

# Vetorizar os dados de texto com N-grams
X_ngrams = ngram_vectorizer.fit_transform(X)

In [13]:
# Com validação cruzada

# Criar uma instância do modelo CatBoost
catboost = CatBoostClassifier(
    task_type="GPU",
    devices="0",
    learning_rate=0.1,
    l2_leaf_reg=5,
    verbose = 50,
    iterations=500,
    used_ram_limit="512mb",
    gpu_ram_part=0.5,
    thread_count=1)  

# Especificar o número de folds para a validação cruzada
num_folds = 5

# Criar um objeto StratifiedKFold para a validação cruzada estratificada
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Lista para armazenar as pontuações de acurácia de cada fold
accuracy_scores = []

# Realizar a validação cruzada
for train_index, test_index in kfold.split(X_ngrams, Y):
    X_train, X_test = X_ngrams[train_index], X_ngrams[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    # Treinar o modelo CatBoost
    catboost.fit(X_train, Y_train)
    
    # Avaliar o modelo com acurácia
    accuracy = catboost.score(X_test, Y_test)
    accuracy_scores.append(accuracy)

# Exibir as pontuações de acurácia
print("Acurácia de cada fold:", accuracy_scores)
print("Acurácia média:", sum(accuracy_scores) / len(accuracy_scores))

In [ ]:
#Matriz de confusão

cm = confusion_matrix(Y_pred, Y_test)

# Plot da matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(Y_test, Y_pred), annot=True, cmap='YlOrBr', fmt='g')

# Adicionando legenda
plt.text(0.5, 0.2, 'Negativos Verdadeiros', color='black', ha='center', va='center')
plt.text(1.5, 0.2, 'Falsos Neutros', color='black', ha='center', va='center')
plt.text(2.5, 0.2, 'Falsos Positivos', color='black', ha='center', va='center')

plt.text(0.5, 1.2, 'Falsos Negativos', color='black', ha='center', va='center')
plt.text(1.5, 1.2, 'Neutros Verdadeiros', color='black', ha='center', va='center')
plt.text(2.5, 1.2, 'Falsos Positivos', color='black', ha='center', va='center')

plt.text(0.5, 2.2, 'Falsos Negativos', color='black', ha='center', va='center')
plt.text(1.5, 2.2, 'Falsos Neutros', color='black', ha='center', va='center')
plt.text(2.5, 2.2, 'Positivos Verdadeiros', color='white', ha='center', va='center')

# Definindo rótulos dos eixos
plt.xlabel('Previsto')
plt.ylabel('Verdadeiro')
plt.title('Matriz de Confusão')

plt.show()

In [ ]:
# Sem validação cruzada 

# Dividir a base de dados em 80% treinamento, 20% teste/validação
X_train, X_test, Y_train, Y_test = train_test_split(X_ngrams, Y, test_size=0.20, random_state=42, stratify=Y)

# Criar uma instância do modelo CatBoost
catboost = CatBoostClassifier(
    task_type="GPU",
    devices="0",
    learning_rate=0.1,
    l2_leaf_reg=5,
    verbose=True,
    used_ram_limit="2gb",
    gpu_ram_part=0.5,
    thread_count=2
    )  # verbose=0 para suprimir a saída durante o treinamento

# Treinar o modelo com os dados de treinamento (80%)
catboost.fit(X_train, Y_train)

In [ ]:
# Avaliar o modelo com os dados de teste (20%)
Y_test_pred = catboost.predict(X_test)
test_accuracy = accuracy_score(Y_test, Y_test_pred)
print("Acurácia no conjunto de teste:", test_accuracy)